<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_Francesca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff

import numpy as np
from numpy import savetxt

import pandas as pd
from pandas import DataFrame
from pandas import read_csv
from pandas import concat

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, LeakyReLU, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib.pyplot import legend

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
def generate_model(shape):
  
  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(113, input_dim=shape, use_bias=False, kernel_initializer='random_uniform'))#, activation='relu'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.5))  
  
  model.add(Dense(229, use_bias=False, kernel_initializer='random_uniform')) #, activation='relu'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
 
  model.add(Dense(153, use_bias=False, kernel_initializer='random_uniform'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
 
  #print(model.summary())

  return model

In [7]:
#LOAD DATA
print("Loading data...")

dataset = '/root/data/6_uHoo_featureDataset.arff'

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("Converting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

Loading data...
DATASET LOADED
Converting values...
CONVERSION DONE


In [8]:
#OPTIMIZERS
sgd = optimizers.SGD(lr=0.0001)
adm = optimizers.Adam(lr=0.0001)
rms = optimizers.RMSprop(lr=0.001)

#OTHER OPTIMIZERS
adagrad = optimizers.Adagrad(lr=0.0001)
adadelta = optimizers.Adadelta(lr=0.0001)
adamax = optimizers.Adamax(lr=0.0001)
nadam = optimizers.Nadam(lr=0.0001)

#DEFINE 10-FOLD CROSS-VALIDATION
kfold = KFold(n_splits=10, shuffle=False)
cvscores = []
predictions = []

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  model = generate_model(X[train].shape[-1])

  #COMPILE MODEL
  model.compile(loss='binary_crossentropy', optimizer = adm , metrics=['accuracy'])
  
  #EARLY STOPPING
  #es = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=1, mode='auto')

  #FIT MODEL
  history = model.fit(X[train], Y[train], epochs=6, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)#, callbacks = [es])

  #EVALUATE MODEL
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 1)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  pred = model.predict_classes(X[test], batch_size=128, verbose=1)
  predictions.append(pred)
  
  y_true = Y[test]
  tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
  (tn, fp, fn, tp)
  other = 100*tn/(tn+fp)
  pasto = 100*tp/(fn+tp)
  print("Other: %.2f %%" % other)
  print("Pasto: %.2f %%" % pasto)
  
  if(i==10): predictions.append(Y[test])
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



FOLD: 1
Epoch 1/6
206937/206937 [==============================] - 19s 91us/step - loss: 0.6616 - acc: 0.6691
Epoch 2/6
206937/206937 [==============================] - 17s 83us/step - loss: 0.5723 - acc: 0.7382
Epoch 3/6
206937/206937 [==============================] - 17s 83us/step - loss: 0.5271 - acc: 0.7518
Epoch 4/6
206937/206937 [==============================] - 17s 81us/step - loss: 0.5035 - acc: 0.7543
Epoch 5/6
206937/206937 [==============================] - 17s 81us/step - loss: 0.4863 - acc: 0.7593
Epoch 6/6
22993/22993 [==============================] - 1s 33us/step
Test loss: 39.88%
Test accuracy: 78.95%
22993/22993 [==============================] - 1s 32us/step
Other: 80.35 %
Pasto: 51.13 %

FOLD: 2
Epoch 1/6
206937/206937 [==============================] - 18s 89us/step - loss: 0.6276 - acc: 0.6964
Epoch 2/6
206937/206937 [==============================] - 17s 83us/step - loss: 0.5454 - acc: 0.7398
Epoch 3/6
206937/206937 [==============================] - 17s 83us/

In [0]:
#model.save('my_model1_cv.h5')

#model = load_model('my_model1_cv.h5')